In [1]:
!pip install -q nltk bert-score
!pip install -q rouge-metric

In [3]:
import pandas as pd

# Load the saved CSV file
generated_summaries_soap_complete = pd.read_csv("summarized_output.csv")

# Verify the data
print(generated_summaries_soap_complete.head())

                                               input  \
0  Good afternoon, champ, how you holding up? Goo...   
1  What brings you in here today? Hi, I'm um, I'm...   
2  Do you have any known allergies to medications...   
3  How may I help you today? Yeah I've had, a fev...   
4  It sounds like that you're experiencing some c...   

                                              output  \
0  Subjective:\n- Symptoms: Lower back pain, radi...   
1  Subjective:\n- Presenting with dry cough for 1...   
2  Subjective:\n- No known allergies to medicatio...   
3  Subjective:\n- Fever and dry cough started 4 d...   
4  Subjective:\n- Presenting with chest pain for ...   

                                     id  \
0  39a26c55-f710-4272-8609-2a725ef6d068   
1  b1448089-3c41-423a-9737-0ed25e0e99c8   
2  78cf9b57-3a1f-41df-ba55-af3bbe843228   
3  44be7957-be23-4cc9-a5e2-cb7139dc2079   
4  3324cd95-1729-4b0b-8916-dba010ac811e   

                                            entities  \
0  {'PROBLE

In [4]:
import pandas as pd
import time
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score
from rouge_metric import PyRouge
import numpy as np

# -----------------------------------------------------
# 1. Text Cleaning
# -----------------------------------------------------
def clean_text(text):
    """Clean and normalize text."""
    if pd.isna(text) or not isinstance(text, str):
        return ""
    return ' '.join(text.strip().lower().split())

# -----------------------------------------------------
# 2. BLEU Computation
# -----------------------------------------------------
def compute_bleu_scores(reference, candidate):
    smoothing_function = SmoothingFunction().method1
    bleu1 = sentence_bleu([reference.split()], candidate.split(), weights=(1.0, 0, 0, 0), smoothing_function=smoothing_function)
    bleu2 = sentence_bleu([reference.split()], candidate.split(), weights=(0.5, 0.5, 0, 0), smoothing_function=smoothing_function)
    return bleu1 * 100, bleu2 * 100

# -----------------------------------------------------
# 3. ROUGE-L Computation
# -----------------------------------------------------
def compute_rouge_l(reference, candidate):
    rouge = PyRouge(rouge_n=(1, 2), rouge_l=True, skip_gap=4)
    scores = rouge.evaluate([candidate], [[reference]])
    return scores['rouge-l']['f'] * 100

# -----------------------------------------------------
# 4. BERTScore in Batches
# -----------------------------------------------------
def compute_bert_score_batched(references, candidates, batch_size=32):
    all_P, all_R, all_F1 = [], [], []
    for i in range(0, len(references), batch_size):
        batch_refs = references[i:i + batch_size]
        batch_cands = candidates[i:i + batch_size]
        P, R, F1 = score(batch_cands, batch_refs, lang="en", verbose=False)
        all_P.extend([p * 100 for p in P.tolist()])
        all_R.extend([r * 100 for r in R.tolist()])
        all_F1.extend([f * 100 for f in F1.tolist()])
    return all_P, all_R, all_F1

# -----------------------------------------------------
# 5. Main Evaluation Function
# -----------------------------------------------------
def evaluate_summaries(df):
    bleu1_scores, bleu2_scores, rouge_l_scores = [], [], []
    print("Computing BLEU and ROUGE-L scores...")

    with tqdm(total=len(df), desc="Processing Rows", unit="row") as pbar:
        for _, row in df.iterrows():
            reference = clean_text(row['output'])
            candidate = clean_text(row['generated_summary'])

            if not reference or not candidate:
                print(f"Empty text - Reference: '{reference}', Candidate: '{candidate}'")
                bleu1_scores.append(0.0)
                bleu2_scores.append(0.0)
                rouge_l_scores.append(0.0)
            else:
                bleu1, bleu2 = compute_bleu_scores(reference, candidate)
                bleu1_scores.append(bleu1)
                bleu2_scores.append(bleu2)
                rouge_l_scores.append(compute_rouge_l(reference, candidate))

            pbar.update(1)

    print("\nComputing BERTScore...")
    references = [clean_text(text) for text in df['output'].tolist()]
    candidates = [clean_text(text) for text in df['generated_summary'].tolist()]

    bert_p, bert_r, bert_f1 = compute_bert_score_batched(references, candidates)

    df['bleu1'] = bleu1_scores
    df['bleu2'] = bleu2_scores
    df['rouge_l'] = rouge_l_scores
    df['bert_p'] = bert_p
    df['bert_r'] = bert_r
    df['bert_f1'] = bert_f1

    print("\nEvaluation Metrics (in percentages):")
    print("Average BLEU-1:", np.mean(bleu1_scores), "%")
    print("Average BLEU-2:", np.mean(bleu2_scores), "%")
    print("Average ROUGE-L:", np.mean(rouge_l_scores), "%")
    print("Average BERT P:", np.mean(bert_p), "%")
    print("Average BERT R:", np.mean(bert_r), "%")
    print("Average BERT F1:", np.mean(bert_f1), "%")

    print("\nStandard Deviations (in percentages):")
    print("BLEU-1 Std:", np.std(bleu1_scores), "%")
    print("BLEU-2 Std:", np.std(bleu2_scores), "%")
    print("ROUGE-L Std:", np.std(rouge_l_scores), "%")
    print("BERT P Std:", np.std(bert_p), "%")
    print("BERT R Std:", np.std(bert_r), "%")
    print("BERT F1 Std:", np.std(bert_f1), "%")

    return df

# -----------------------------------------------------
# 6. Load CSV & Evaluate
# -----------------------------------------------------
if __name__ == "__main__":
    # Load the CSV with 'output' and 'generated_summary'
    df_summaries = pd.read_csv("summarized_output.csv")
    print(df_summaries.head())

    # Evaluate
    results_df = evaluate_summaries(df_summaries)

    # Save the DataFrame with the metrics
    results_df.to_csv("soap_evaluation_results.csv", index=False)
    print("\nResults saved to 'soap_evaluation_results.csv'")


                                               input  \
0  Good afternoon, champ, how you holding up? Goo...   
1  What brings you in here today? Hi, I'm um, I'm...   
2  Do you have any known allergies to medications...   
3  How may I help you today? Yeah I've had, a fev...   
4  It sounds like that you're experiencing some c...   

                                              output  \
0  Subjective:\n- Symptoms: Lower back pain, radi...   
1  Subjective:\n- Presenting with dry cough for 1...   
2  Subjective:\n- No known allergies to medicatio...   
3  Subjective:\n- Fever and dry cough started 4 d...   
4  Subjective:\n- Presenting with chest pain for ...   

                                     id  \
0  39a26c55-f710-4272-8609-2a725ef6d068   
1  b1448089-3c41-423a-9737-0ed25e0e99c8   
2  78cf9b57-3a1f-41df-ba55-af3bbe843228   
3  44be7957-be23-4cc9-a5e2-cb7139dc2079   
4  3324cd95-1729-4b0b-8916-dba010ac811e   

                                            entities  \
0  {'PROBLE

Processing Rows: 100%|██████████| 100/100 [00:03<00:00, 25.29row/s]



Computing BERTScore...


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You sho


Evaluation Metrics (in percentages):
Average BLEU-1: 11.545791495966942 %
Average BLEU-2: 6.085986052986498 %
Average ROUGE-L: 10.698332475527389 %
Average BERT P: 80.50110596418381 %
Average BERT R: 84.0757542848587 %
Average BERT F1: 82.24017286300659 %

Standard Deviations (in percentages):
BLEU-1 Std: 5.491333362619107 %
BLEU-2 Std: 4.177290634566192 %
ROUGE-L Std: 4.468480836822641 %
BERT P Std: 1.6490731969112913 %
BERT R Std: 2.9839974015914583 %
BERT F1 Std: 2.21179424193905 %

Results saved to 'soap_evaluation_results.csv'
